# AI Computer Assignment 2 (Genetic Algorithms)
Mohammad Saadati - 
_810198410_

## Introduction
Goal of this project is to create genetic algorithm which will solve sudoku.

In [69]:
import numpy
import random
import re
from math import floor
import string
import time

POPULATION_SIZE = 100  
NUMBER_OF_GENERATIONS = 1500 
MUTATION_PROBABILITY = 0.08
CROSSOVER_PROBABILITY = 0.3
MAX_REPEAT_COUNT = 100
ALPHABET = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p',
            'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

ord('Z')

90

## Phase 0:

In [61]:
class TextProcessor(object):
    def __init__(self, _initial_text):
        self.text = _initial_text
    
    def lower_text(self, text):
        return (text).lower()
    
    def split_text(self, text):
        return re.split('[^a-zA-Z]', text)
    
    def make_dict(self, words):
        return set(words)
    
    def clean_text(self, text):
        words = self.split_text(text)
        return self.make_dict(words)

## Phase 1:
**gene**: each row represents a gene.

**chromosome**: a sequence of gates(genes) represent a chromosome.

In [62]:
class Chromosome(object):
    def __init__(self, _chromosome):
        self.fitness = 0
        self.chromosome = _chromosome
    
    def get_key(self, index):
        return self.chromosome[index]
    
    def update_fitness(self, fitness):
        self.fitness = fitness
            
    def mutation(self, mutation_rate, intial_input):
        None
        

## Phase 2:
Initialized population of size 200 randomly using the possible_values.

## Phase 3:
Fitness defiend in this problem is the accuracy of our prediction for each chromosome. The fitness of a chromosome solution is determined by how close it is to being the actual solution to the puzzle.

## Phase 4:
### Mutation and Crossover 

In [86]:
class Decoder(object):
    def __init__(self, global_text, encoded_text, _key_lenght = 14):
        self.text_processor = TextProcessor(global_text)
        self.global_text_dict = self.text_processor.clean_text(self.text_processor.lower_text(global_text))
        self.encoded_text = encoded_text
        self.encoded_text_word = self.text_processor.split_text(self.text_processor.lower_text(encoded_text))
        self.encoded_text_word = [w for w in self.encoded_text_word if w != '']
        self.key_lenght = _key_lenght
        self.population = []
        self.best_fitness = -1
        self.best_chromosome = None
        
    def initiate_population(self):
        population = []
        for i in range(POPULATION_SIZE):
            key = random.sample(ALPHABET, self.key_lenght)
            while key in population:
                random.shuffle(key)
            self.population.append(Chromosome(''.join([str(elem) for elem in key])))
        for i in range(POPULATION_SIZE):
            self.population[i].update_fitness(self.calculate_fitness(self.population[i].chromosome))
            
    def get_fitness(self, chromosome: Chromosome):
        return chromosome.fitness
    
    def calculate_fitness(self, key):
        counter = 0
        decoded_words = []
        for word in self.encoded_text_word:
            decoded_word = ""
            for letter in word:
                cur_key = ord(key[counter]) - 97
                letter = ord(letter) - 97
                decoded_word += chr(((letter - cur_key) % 26) + 97)
                counter = (counter + 1) % self.key_lenght
            decoded_words.append(decoded_word)
        is_in_dict = 0
        for w in decoded_words:
            if w in self.global_text_dict:
                is_in_dict += 1
        return is_in_dict / len(decoded_words)
    
    def evaluate_fitnesses(self):
        for i in range(POPULATION_SIZE):
            self.population[i].update_fitness(self.calculate_fitness(self.population[i].chromosome))   
            
    def crossover(self, first_chromosome, second_chromosome):
        p = floor(self.key_lenght / 2)
        new_key1 = first_chromosome.chromosome[:p] + second_chromosome.chromosome[p:]
        new_key2 = first_chromosome.chromosome[p:] + second_chromosome.chromosome[:p]
        result1 = Chromosome(new_key1)
        result1.fitness = self.calculate_fitness(new_key1)
        result2 = Chromosome(new_key2)
        result2.fitness = self.calculate_fitness(new_key2)
        return result1, result2
    
    def mutate(self, chromosome):
        result = chromosome
        new_chromosome_key = chromosome.chromosome
        if random.random() < MUTATION_PROBABILITY:
            p = random.randint(0, self.key_lenght - 1)
            candidate = new_chromosome_key[:p] + random.choice(string.ascii_lowercase) + new_chromosome_key[p+1:]
            candidate_fitness = self.calculate_fitness(candidate)
            if candidate_fitness > chromosome.fitness:
                result = Chromosome(candidate)
                result.fitness = candidate_fitness
        return result   
    
    def decode_text(self, key):
        counter = 0
        decoded_text = ""
        for i in range(len(self.encoded_text)):
            if ord(self.encoded_text[i]) >= 97 and ord(self.encoded_text[i]) <= 122:
                cur_key = ord(key[counter]) - 97
                encoded_letter = ord(self.encoded_text[i]) - 97
                decoded_text += chr(((encoded_letter - cur_key) % 26) + 97)
                counter = (counter + 1) % self.key_lenght
            elif ord(self.encoded_text[i]) >= 65 and ord(self.encoded_text[i]) <= 90:
                cur_key = ord(key[counter]) - 97
                encoded_letter = ord(self.encoded_text[i]) - 65
                decoded_text += chr(((encoded_letter - cur_key) % 26) + 65)
                counter = (counter + 1) % self.key_lenght
            else:
                decoded_text += self.encoded_text[i]
 
        return decoded_text
         
    def decode(self):
        start = time.time()
        generation = 0
        repeat_count = 0
        self.initiate_population()
        for i in range(NUMBER_OF_GENERATIONS):
            print("------------------------------------------------")
            self.population = sorted(self.population, key=self.get_fitness, reverse=True)
            self.best_fitness = self.population[0].fitness
            self.best_chromosome = self.population[0].chromosome
            print("Generation %d" % generation)
            if self.best_fitness == 1:
                print("Solution found!")
                print("\nKey:", self.best_chromosome)
                print("\nThe time duration was:",int((time.time() - start)/60), "min and", '%.2f'%((time.time() - start)%60), " secs")
                break
            print("Key:", self.best_chromosome, " Fitness: %f" % self.best_fitness)
            
            elites_num = floor(CROSSOVER_PROBABILITY * POPULATION_SIZE) 
            elites = self.population[:elites_num]
            new_pop = [self.mutate(chromosome) for chromosome in elites]
            crossovering = [chromosome for chromosome in self.population[:(POPULATION_SIZE - elites_num)]]
            random.shuffle(crossovering)
            
            while(len(new_pop) != POPULATION_SIZE):
                fi_par = crossovering.pop(0)
                sec_par = crossovering.pop(0)
                fi_ch, sec_ch = self.crossover(fi_par, sec_par)
                fi_ch = self.mutate(fi_ch)
                sec_ch = self.mutate(sec_ch)
                new_pop.append(fi_ch)
                new_pop.append(sec_ch)
                
            self.population = new_pop
            generation+=1
            
        return self.decode_text(self.best_chromosome)

encodedText = open("encoded_text.txt").read()
globalText = open("global_text.txt").read()
d = Decoder(globalText, encodedText)
decodedText = d.decode()

print("\nThe decoded text is:\n")
print(decodedText)

------------------------------------------------
Generation 0
Key: jvgyehrtuiwnma  Fitness: 0.056711
------------------------------------------------
Generation 1
Key: jevzcsooirfyvn  Fitness: 0.062382
------------------------------------------------
Generation 2
Key: jevzcsooirfyvn  Fitness: 0.062382
------------------------------------------------
Generation 3
Key: hzdwouerbfydag  Fitness: 0.066163
------------------------------------------------
Generation 4
Key: yidhmoktwrfeio  Fitness: 0.069943
------------------------------------------------
Generation 5
Key: zubcselrbfydan  Fitness: 0.071834
------------------------------------------------
Generation 6
Key: zubcselrbfydan  Fitness: 0.071834
------------------------------------------------
Generation 7
Key: zubcselrbfydan  Fitness: 0.071834
------------------------------------------------
Generation 8
Key: zubcselrbfydan  Fitness: 0.071834
------------------------------------------------
Generation 9
Key: zubcselrbfydan  Fitness:

------------------------------------------------
Generation 81
Key: vbtfrrxthsteiv  Fitness: 0.162571
------------------------------------------------
Generation 82
Key: vbtfrrxthsteiv  Fitness: 0.162571
------------------------------------------------
Generation 83
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 84
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 85
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 86
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 87
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 88
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 89
Key: vbtfroxthsteiv  Fitness: 0.168242
------------------------------------------------
Generation 90
Key: vbtfroxthsteiv

------------------------------------------------
Generation 161
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 162
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 163
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 164
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 165
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 166
Key: abterounhstein  Fitness: 0.257089
------------------------------------------------
Generation 167
Key: abcerounhstein  Fitness: 0.260870
------------------------------------------------
Generation 168
Key: abcerounhstein  Fitness: 0.260870
------------------------------------------------
Generation 169
Key: abcerounhstein  Fitness: 0.260870
------------------------------------------------
Generation 170
Key: abce

------------------------------------------------
Generation 245
Key: albertjnnstein  Fitness: 0.601134
------------------------------------------------
Generation 246
Key: albertjnnstein  Fitness: 0.601134
------------------------------------------------
Generation 247
Key: albertjnnstein  Fitness: 0.601134
------------------------------------------------
Generation 248
Key: albertcynstein  Fitness: 0.604915
------------------------------------------------
Generation 249
Key: albertkynstein  Fitness: 0.608696
------------------------------------------------
Generation 250
Key: albertkynstein  Fitness: 0.608696
------------------------------------------------
Generation 251
Key: albertkynstein  Fitness: 0.608696
------------------------------------------------
Generation 252
Key: albertkynstein  Fitness: 0.608696
------------------------------------------------
Generation 253
Key: albertkynstein  Fitness: 0.608696
------------------------------------------------
Generation 254
Key: albe

Mutate a chromosome by picking a row, and then picking two values within that row to swap.

The crossover function requires two parents to be selected from the population pool. The Tournament class is used to do this. Pick 2 random chromosomes from the population and get them to compete against each other.

Crossover relates to the analogy of genes within each parent chromosome mixing together in the hopes of creating a fitter child chromosome. Create two new child chromosomes by crossing over parent genes. 

## Phase 5:
Load a file to solve. 

For each generation, calculated the fitness of each candidate and sorted them. Then traverse all the chromosomes to check for solution. if solution not found, select the fittest chromosomes and preserve them for the next generation(120 elites in new generation). Took first 120 as elites(elites would definately be part of the next generation). For the remaining 80, randomly selected two candidates from the population and performed crossover among them. Mutation is done on each candidate. Create the rest of the candidates and select parents from population via a tournament. Then append elites into the end of the population. These will not have been affected by crossover or mutation. Calculate new adaptive mutation rate. We continue doing this until we get a fitness value of 1. This indicates we have successfully found a solution to the sudoku problem. To avoid biasing chromosomes, we can Re-generate the population if for example 100 generations have passed with the fittest two chromosomes always having the same fitness.

## Phase 6: 
### Questions
#### 1.
We choose tournament selection because it can be implemented efficiently as no sorting of the population is not required.
#### 2.
Fitness function should be sufficiently fast to compute and must quantitatively measure how fit a given solution is or how fit individuals can be produced from the given solution. Our fitness function satisfies these criteria and its suitable for this problem.

#### 3.
**cross over** : This makes it possible that if part of the selected genes are suitable and part is inappropriate, the improper part is likely to be discarded, thus combining the part that we think is appropriate with another part of the chromosome that brings us closer to the answer. We consider 0.85 for the crossover probability and it is constant during the execution of the algorithm.

**mutation** : This operation is performed after each crossover, because we may be stuck in a certain state every time after this operation. By slightly changing these chromosomes, we can get out of this state and get closer to the answer. we have chosen a variable mutationRate which is the mutation probabality and its default value is 0.5 to make more change to chromosomes. Whenever we are close to the solution, we decrease it by 0.002 and whenever we are in a local maximum, we increase it by 0.002.
#### 4.
If this happens to us, it means that we are stuck at a local maximum point and we have to get out of this point. To prevent chromosomes stability, when maximum fitness of our model gets below 90% we increase mutation and crossover probability to cause more changes and if maximum fitness of our model remains the same for a number of generation we increase mutation and crossover probability to cause more changes. Also we can store history of generations and if we are repeating on a fitness for example for 100 times Then i change the probabilty of mutationRate for one generation to get out of this situation. 